#### 构建Agent ####

In [1]:
import pandas as pd
import json
from agents.standard_designer import standard_designer
from agents.web_explorer import web_explorer
from agents.evaluator import evaluator
from agents.reporter import reporter
from crewai import Task, Crew

# 1. 读取网页数据（从CSV）
df = pd.read_csv('data/structured_urls.csv')
sample_rows = df.head(5).to_dict(orient='records')  # 前5条用于测试

# 2. 设定标准制定任务（只需做一次）
example_urls = "\n".join([item['url'] for item in sample_rows])
task_define_standard = Task(
    description=f"""请访问以下网页，观察其中的用户互动要素，并总结网页“触点丰富度”的评估标准（如联系方式、二维码、客服等）。
网页列表：
{example_urls}

请输出：
- 建议评估维度（3-6个）
- 每个维度的判断方式（出现、明显程度）
- 最终打分机制建议
""",
    expected_output="一份结构化评估标准，格式为JSON，包含维度名、定义、评分方式",
    agent=standard_designer
)

# 3. 构建其他网页处理任务列表
tasks = [task_define_standard]
for item in sample_rows:
    url = item['url']
    domain = item['domain']
    year = item['year']

    # 网页内容提取任务
    task_extract = Task(
        description=f"访问网页：{url}，提取该页面的主要内容",
        expected_output="返回该网页中可读文本内容的字符串（不要包含HTML标签）",
        agent=web_explorer
    )

    # 网页内容评估任务
    task_evaluate = Task(
        description=(
            f"请根据由标准制定专家提供的评估维度，对以下网页内容进行分析，输出该网页的触点丰富度评分报告。\n"
            f"- 网页来源：{domain}（{year}）\n"
            f"- 网页地址：{url}\n"
            f"网页内容将通过上一步生成的文本提供。"
        ),
        expected_output=(
            "请返回结构化JSON，包括：\n"
            "- domain\n- year\n- url\n- score\n"
            "- summary（评价摘要）\n- touchpoints（每个维度的具体判定结果）"
        ),
        agent=evaluator
    )

    tasks.extend([task_extract, task_evaluate])

# 报告整合任务（后续执行,只做一次）
task_report = Task(
    description="请收集所有网页的评估结果，并整合成一份整体评估报告，包括汇总统计、整体趋势、共性问题和优秀实践。",
    expected_output="一份结构化JSON格式的报告，包含所有网页评估结果及总结。",
    agent=reporter
)

tasks.append(task_report)

# 4. 创建 Crew 并运行
crew = Crew(
    agents=[standard_designer, web_explorer, evaluator, reporter],
    tasks=tasks,
    verbose=True
)

result = crew.kickoff()

/home/allen2025/crew_touchpoint_evaluator/llm_config.py:10: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  shared_llm = ChatOpenAI(


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 2c3040ca-48d5-4340-ae1d-13d816091998                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 2d1db060-458c-4392-bbf7-15032f109965
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 2d1db060-458c-4392-bbf7-15032f109965
       Status: Executing Task...
    └── 🤖 Agent: 网页触点评估标准设计师
            Status: In Progress

# Agent: 网页触点评估标准设计师
## Task: 请访问以下网页，观察其中的用户互动要素，并总结网页“触点丰富度”的评估标准（如联系方式、二维码、客服等）。
网页列表：
https://web.archive.org/web/20090604043333/http://www.1000bulbs.com:80/
https://web.archive.org/web/20101231143820/http://www.1000bulbs.com/
https://web.archive.org/web/20111228071456/http://www.1000bulbs.com:80/
https://web.archive.org/web/20121226235104/http://www.1000bulbs.com:80/
https://web.archive.org/web/20131231104800/http://www.1000bulbs.com/

请输出：
- 建议评估维度（3-6个）
- 每个维度的判断方式（出现、明显程度）
- 最终打分机制建议



...

# Agent: 评估报告生成器
## Task: 请收集所有网页的评估结果，并整合成一份整体评估报告，包括汇总统计、整体趋势、共性问题和优秀实践。


# Agent: 评估报告生成器
## Final Answer: 
```json
{
  "evaluationReport": {
    "domain": "1000bulbs.com",
    "overallSummary": {
      "totalEvaluations": 6,
      "averageScore": 9,
      "overallTrend": "触点丰富度一般，未见显著提升或下降的趋势。",
      "commonIssues": [
        "社交媒体链接缺乏，未能有效连接用户。",
        "互动元素较少，用户参与感不足。",
        "缺乏二维码及应用程序的相关信息，未能满足移动用户的需求。"
      ],
      "bestPractices": [
        "提供了多种联系方式，便于客户联系。",
        "客服支持选项较为全面，确保客户疑问得到解答。"
      ]
    },
    "evaluations": [
      {
        "year": 2009,
        "url": "https://web.archive.org/web/20090604043333/http://www.1000bulbs.com:80/",
        "score": 10,
        "summary": "网页触点丰富度一般，提供了一定数量的联系方式和客服支持选项，但社交媒体链接和互动元素相对较少，缺乏二维码及应用程序的相关信息。",
        "touchpoints": {
          "联系方式": {
            "appearance": 1,
            "visibility": 1
          },
          "社交媒体链接": {
            "appearance": 0,
            "visibility": 0
          },
          "客

🚀 Crew: crew
├── 📋 Task: 2d1db060-458c-4392-bbf7-15032f109965
│      Assigned to: 网页触点评估标准设计师
│      Status: ✅ Completed
│   └── 🤖 Agent: 网页触点评估标准设计师
│           Status: ✅ Completed
├── 📋 Task: 95a9b893-0010-461f-871e-422cdd3f1296
│      Assigned to: 网页提取专家
│      Status: ✅ Completed
│   └── 🤖 Agent: 网页提取专家
│           Status: ✅ Completed
├── 📋 Task: ddf2c082-57dd-4595-8fa2-4007bd320f52
│      Assigned to: 网页触点评估员
│      Status: ✅ Completed
│   └── 🤖 Agent: 网页触点评估员
│           Status: ✅ Completed
├── 📋 Task: 237a5fc1-9cf9-4565-aa5e-85ac27432848
│      Assigned to: 网页提取专家
│      Status: ✅ Completed
│   └── 🤖 Agent: 网页提取专家
│           Status: ✅ Completed
├── 📋 Task: 550209bc-accf-482b-8de7-30a6b404fafb
│      Assigned to: 网页触点评估员
│      Status: ✅ Completed
│   └── 🤖 Agent: 网页触点评估员
│           Status: ✅ Completed
├── 📋 Task: e785d3b0-b640-47dc-9f2c-df0797c96626
│      Assigned to: 网页提取专家
│      Status: ✅ Completed
│   └── 🤖 Agent: 网页提取专家
│           Status: ✅ Completed
├── 📋 Task: a67fb474-6007-4817-a26b-d094a3dc4a80
│      Assigned to: 网页触点评估员
│      Status: ✅ Completed
│   └── 🤖 Agent: 网页触点评估员
│           Status: ✅ Completed
├── 📋 Task: afa229b0-8db6-4393-b1ef-1d8ae3ac886e
│      Assigned to: 网页提取专家
│      Status: ✅ Completed
│   └── 🤖 Agent: 网页提取专家
│           Status: ✅ Completed
├── 📋 Task: 0f619858-fa8c-4551-a689-eb28cd5bb0c0
│      Assigned to: 网页触点评估员
│      Status: ✅ Completed
│   └── 🤖 Agent: 网页触点评估员
│           Status: ✅ Completed
├── 📋 Task: 4c06ace7-348d-467c-adab-19569c1547cd
│      Assigned to: 网页提取专家
│      Status: ✅ Completed
│   └── 🤖 Agent: 网页提取专家
│           Status: ✅ Completed
├── 📋 Task: 9d8686b3-6d50-45cd-9136-ecf15c790513
│      Assigned to: 网页触点评估员
│      Status: ✅ Completed
│   └── 🤖 Agent: 网页触点评估员
│           Status: ✅ Completed
└── 📋 Task: a5d0688e-8378-4ba0-a9e7-2f899e9f7523
       Status: Executing Task...
    └── 🤖 Agent: 评估报告生成器
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 2d1db060-458c-4392-bbf7-15032f109965
│      Assigned to: 网页触点评估标准设计师
│      Status: ✅ Completed
│   └── 🤖 Agent: 网页触点评估标准设计师
│           Status: ✅ Completed
├── 📋 Task: 95a9b893-0010-461f-871e-422cdd3f1296
│      Assigned to: 网页提取专家
│      Status: ✅ Completed
│   └── 🤖 Agent: 网页提取专家
│           Status: ✅ Completed
├── 📋 Task: ddf2c082-57dd-4595-8fa2-4007bd320f52
│      Assigned to: 网页触点评估员
│      Status: ✅ Completed
│   └── 🤖 Agent: 网页触点评估员
│           Status: ✅ Completed
├── 📋 Task: 237a5fc1-9cf9-4565-aa5e-85ac27432848
│      Assigned to: 网页提取专家
│      Status: ✅ Completed
│   └── 🤖 Agent: 网页提取专家
│           Status: ✅ Completed
├── 📋 Task: 550209bc-accf-482b-8de7-30a6b404fafb
│      Assigned to: 网页触点评估员
│      Status: ✅ Completed
│   └── 🤖 Agent: 网页触点评估员
│           Status: ✅ Completed
├── 📋 Task: e785d3b0-b640-47dc-9f2c-df0797c96626
│      Assigned to: 网页提取专家
│      Status: ✅ Completed
│   └── 🤖 Agent: 网页提取专家
│           Status: ✅ Completed
├── 📋 Task: a67fb474-6007-4817-a26b-d094a3dc4a80
│      Assigned to: 网页触点评估员
│      Status: ✅ Completed
│   └── 🤖 Agent: 网页触点评估员
│           Status: ✅ Completed
├── 📋 Task: afa229b0-8db6-4393-b1ef-1d8ae3ac886e
│      Assigned to: 网页提取专家
│      Status: ✅ Completed
│   └── 🤖 Agent: 网页提取专家
│           Status: ✅ Completed
├── 📋 Task: 0f619858-fa8c-4551-a689-eb28cd5bb0c0
│      Assigned to: 网页触点评估员
│      Status: ✅ Completed
│   └── 🤖 Agent: 网页触点评估员
│           Status: ✅ Completed
├── 📋 Task: 4c06ace7-348d-467c-adab-19569c1547cd
│      Assigned to: 网页提取专家
│      Status: ✅ Completed
│   └── 🤖 Agent: 网页提取专家
│           Status: ✅ Completed
├── 📋 Task: 9d8686b3-6d50-45cd-9136-ecf15c790513
│      Assigned to: 网页触点评估员
│      Status: ✅ Completed
│   └── 🤖 Agent: 网页触点评估员
│           Status: ✅ Completed
└── 📋 Task: a5d0688e-8378-4ba0-a9e7-2f899e9f7523
       Assigned to: 评估报告生成器
       Status: ✅ Completed
    └── 🤖 Agent: 评估报告生成器
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: a5d0688e-8378-4ba0-a9e7-2f899e9f7523                                                                     │
│  Agent: 评估报告生成器                                                                                          │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 2c3040ca-48d5-4340-ae1d-13d816091998                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

#### 获取结果 ####

In [ ]:
# 5. 输出结果保存为 JSON 文件
with open("result.json", "w", encoding="utf-8") as f:
    # json.dump(result, f, indent=2, ensure_ascii=False)
    json.dump({"result": str(result.output)}, f, indent=2, ensure_ascii=False)

print("✅ 分析完成，结果已保存到 result.json")